# AVALON Nuclear AI Crisis - Data Analysis
## Imperial College London - Claude Hacks

**Objective**: Analyze AVALON's misaligned behavior and build models to predict true nuclear risk vs AVALON's biased recommendations.

**Problem Statement**: AVALON overreacts to public anxiety, social media rumors, and regulatory scrutiny instead of focusing on true physical risk. We will:
1. Identify patterns in AVALON's decision-making biases
2. Build predictive models for true risk and incidents
3. Compare our models to AVALON's flawed recommendations
4. Provide actionable insights for operators

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Plotting settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
%matplotlib inline

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print('Libraries loaded successfully!')

In [ ]:
# Load data
df = pd.read_csv('avalon_nuclear.csv')

print(f"Dataset Shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
print(f"\nFirst 5 rows:")
df.head()

In [ ]:
# Quick data overview
print("=== DATA OVERVIEW ===")
print(f"Total observations: {len(df):,}")
print(f"Total features: {df.shape[1]}")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"\nData types:")
print(df.dtypes.value_counts())
print(f"\nMemory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

## 2. Exploratory Data Analysis (EDA)

### 2.1 Target Variables Distribution

In [ ]:
# Analyze target variables
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('Target Variables Distribution Analysis', fontsize=16, fontweight='bold')

# True Risk Level
true_risk_counts = df['true_risk_level'].value_counts().sort_index()
axes[0, 0].bar(true_risk_counts.index, true_risk_counts.values, color='steelblue', alpha=0.7)
axes[0, 0].set_title('True Risk Level Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Risk Level (0=Low, 3=Very High)')
axes[0, 0].set_ylabel('Count')
for i, v in enumerate(true_risk_counts.values):
    axes[0, 0].text(i, v + 50, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontweight='bold')

# Incident Occurred
incident_counts = df['incident_occurred'].value_counts()
axes[0, 1].bar(['No Incident', 'Incident'], incident_counts.values, color=['green', 'red'], alpha=0.7)
axes[0, 1].set_title('Actual Incidents Distribution', fontweight='bold')
axes[0, 1].set_ylabel('Count')
for i, v in enumerate(incident_counts.values):
    axes[0, 1].text(i, v + 50, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontweight='bold')

# AVALON Evacuation Recommendation
evac_counts = df['avalon_evac_recommendation'].value_counts()
axes[0, 2].bar(['No Evac', 'Evacuate'], evac_counts.values, color=['lightblue', 'orange'], alpha=0.7)
axes[0, 2].set_title('AVALON Evacuation Recommendations', fontweight='bold')
axes[0, 2].set_ylabel('Count')
for i, v in enumerate(evac_counts.values):
    axes[0, 2].text(i, v + 50, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontweight='bold')

# AVALON Shutdown Recommendation
shutdown_counts = df['avalon_shutdown_recommendation'].value_counts()
axes[1, 0].bar(['No Shutdown', 'Shutdown'], shutdown_counts.values, color=['lightgreen', 'darkred'], alpha=0.7)
axes[1, 0].set_title('AVALON Shutdown Recommendations', fontweight='bold')
axes[1, 0].set_ylabel('Count')
for i, v in enumerate(shutdown_counts.values):
    axes[1, 0].text(i, v + 50, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontweight='bold')

# Human Override
override_counts = df['human_override'].value_counts()
axes[1, 1].bar(['No Override', 'Override'], override_counts.values, color=['gray', 'purple'], alpha=0.7)
axes[1, 1].set_title('Human Overrides', fontweight='bold')
axes[1, 1].set_ylabel('Count')
for i, v in enumerate(override_counts.values):
    axes[1, 1].text(i, v + 50, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontweight='bold')

# Sensor Anomaly
sensor_counts = df['sensor_anomaly_flag'].value_counts()
axes[1, 2].bar(['No Anomaly', 'Anomaly'], sensor_counts.values, color=['lightgray', 'yellow'], alpha=0.7)
axes[1, 2].set_title('Sensor Anomalies', fontweight='bold')
axes[1, 2].set_ylabel('Count')
for i, v in enumerate(sensor_counts.values):
    axes[1, 2].text(i, v + 50, f'{v}\n({v/len(df)*100:.1f}%)', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n⚠️ KEY OBSERVATION: AVALON recommends shutdown in 70% of cases, but only 13% result in actual incidents!")

### 2.2 AVALON's Performance Analysis

In [ ]:
# Confusion matrix style analysis
print("=== AVALON SHUTDOWN RECOMMENDATION vs ACTUAL INCIDENTS ===")
print(pd.crosstab(df['avalon_shutdown_recommendation'], df['incident_occurred'], 
                  rownames=['AVALON Shutdown'], colnames=['Actual Incident'], margins=True))

print("\n=== AVALON EVACUATION RECOMMENDATION vs ACTUAL INCIDENTS ===")
print(pd.crosstab(df['avalon_evac_recommendation'], df['incident_occurred'], 
                  rownames=['AVALON Evac'], colnames=['Actual Incident'], margins=True))

# Calculate AVALON's accuracy
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("\n=== AVALON SHUTDOWN PERFORMANCE METRICS ===")
print(f"Accuracy: {accuracy_score(df['incident_occurred'], df['avalon_shutdown_recommendation']):.3f}")
print(f"Precision: {precision_score(df['incident_occurred'], df['avalon_shutdown_recommendation']):.3f}")
print(f"Recall: {recall_score(df['incident_occurred'], df['avalon_shutdown_recommendation']):.3f}")
print(f"F1-Score: {f1_score(df['incident_occurred'], df['avalon_shutdown_recommendation']):.3f}")

# False positives analysis
false_positives = ((df['avalon_shutdown_recommendation'] == 1) & (df['incident_occurred'] == 0)).sum()
false_negatives = ((df['avalon_shutdown_recommendation'] == 0) & (df['incident_occurred'] == 1)).sum()

print(f"\n⚠️ False Positives (unnecessary shutdowns): {false_positives} ({false_positives/len(df)*100:.1f}%)")
print(f"⚠️ False Negatives (missed incidents): {false_negatives} ({false_negatives/len(df)*100:.1f}%)")

### 2.3 Geographic Distribution

In [ ]:
# Country analysis
country_stats = df.groupby('country').agg({
    'incident_occurred': ['count', 'sum', 'mean'],
    'true_risk_level': 'mean',
    'avalon_shutdown_recommendation': 'mean'
}).round(3)

country_stats.columns = ['Total_Plants', 'Incidents', 'Incident_Rate', 'Avg_True_Risk', 'AVALON_Shutdown_Rate']
country_stats = country_stats.sort_values('Incidents', ascending=False)

print("=== TOP 15 COUNTRIES BY INCIDENT COUNT ===")
print(country_stats.head(15))

# Visualization
fig, axes = plt.subplots(1, 2, figsize=(18, 6))

top_countries = country_stats.head(10)
axes[0].barh(top_countries.index, top_countries['Incidents'], color='crimson', alpha=0.7)
axes[0].set_xlabel('Number of Incidents')
axes[0].set_title('Top 10 Countries by Incidents', fontweight='bold')
axes[0].invert_yaxis()

axes[1].scatter(country_stats['Avg_True_Risk'], country_stats['AVALON_Shutdown_Rate'], 
                s=country_stats['Total_Plants']*2, alpha=0.6, c='blue')
axes[1].set_xlabel('Average True Risk Level')
axes[1].set_ylabel('AVALON Shutdown Rate')
axes[1].set_title('True Risk vs AVALON Shutdown Rate by Country', fontweight='bold')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.4 Technical Features Analysis

In [ ]:
# Key technical features
technical_features = [
    'core_temp_c', 'coolant_pressure_bar', 'neutron_flux', 
    'control_rod_position_pct', 'coolant_flow_rate',
    'radiation_inside_uSv', 'radiation_outside_uSv'
]

# Compare distributions for incidents vs no incidents
fig, axes = plt.subplots(3, 3, figsize=(18, 14))
axes = axes.flatten()
fig.suptitle('Technical Features: Incidents vs No Incidents', fontsize=16, fontweight='bold')

for idx, feature in enumerate(technical_features):
    incident_data = df[df['incident_occurred'] == 1][feature]
    no_incident_data = df[df['incident_occurred'] == 0][feature]
    
    axes[idx].hist(no_incident_data, bins=30, alpha=0.6, label='No Incident', color='green', density=True)
    axes[idx].hist(incident_data, bins=30, alpha=0.6, label='Incident', color='red', density=True)
    axes[idx].set_title(feature, fontweight='bold')
    axes[idx].legend()
    axes[idx].set_ylabel('Density')

# Remove extra subplots
for idx in range(len(technical_features), len(axes)):
    fig.delaxes(axes[idx])

plt.tight_layout()
plt.show()

# Statistical tests
print("\n=== STATISTICAL SIGNIFICANCE (t-test) ===")
print("Testing if technical features differ between incident/no-incident cases:\n")
for feature in technical_features:
    incident_data = df[df['incident_occurred'] == 1][feature]
    no_incident_data = df[df['incident_occurred'] == 0][feature]
    t_stat, p_value = stats.ttest_ind(incident_data, no_incident_data)
    significance = "***" if p_value < 0.001 else "**" if p_value < 0.01 else "*" if p_value < 0.05 else "n.s."
    print(f"{feature:30s} p-value: {p_value:.4f} {significance}")

### 2.5 AVALON Bias Analysis - Social vs Physical Signals

In [ ]:
# Define feature groups
physical_risk_features = [
    'core_temp_c', 'coolant_pressure_bar', 'neutron_flux',
    'radiation_inside_uSv', 'radiation_outside_uSv', 'maintenance_score'
]

social_bias_features = [
    'public_anxiety_index', 'social_media_rumour_index', 'regulator_scrutiny_score'
]

# Correlation with AVALON decisions
print("=== CORRELATION WITH AVALON SHUTDOWN RECOMMENDATION ===")
print("\nPhysical Risk Features:")
for feature in physical_risk_features:
    corr = df[feature].corr(df['avalon_shutdown_recommendation'])
    print(f"{feature:30s}: {corr:+.3f}")

print("\nSocial Bias Features (suspected over-weighting):")
for feature in social_bias_features:
    corr = df[feature].corr(df['avalon_shutdown_recommendation'])
    print(f"{feature:30s}: {corr:+.3f}")

print("\n=== CORRELATION WITH TRUE RISK LEVEL ===")
print("\nPhysical Risk Features:")
for feature in physical_risk_features:
    corr = df[feature].corr(df['true_risk_level'])
    print(f"{feature:30s}: {corr:+.3f}")

print("\nSocial Bias Features:")
for feature in social_bias_features:
    corr = df[feature].corr(df['true_risk_level'])
    print(f"{feature:30s}: {corr:+.3f}")

In [ ]:
# Visualization of bias
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('AVALON Bias: Social Features vs Physical Risk', fontsize=16, fontweight='bold')

for idx, feature in enumerate(social_bias_features):
    # AVALON shutdown vs social feature
    axes[0, idx].scatter(df[feature], df['avalon_shutdown_recommendation'], 
                         alpha=0.3, c='red', s=10)
    axes[0, idx].set_xlabel(feature)
    axes[0, idx].set_ylabel('AVALON Shutdown')
    axes[0, idx].set_title(f'{feature} vs AVALON Decision', fontsize=10, fontweight='bold')
    
    # True risk vs social feature
    axes[1, idx].scatter(df[feature], df['true_risk_level'], 
                         alpha=0.3, c='blue', s=10)
    axes[1, idx].set_xlabel(feature)
    axes[1, idx].set_ylabel('True Risk Level')
    axes[1, idx].set_title(f'{feature} vs True Risk', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 If social features correlate strongly with AVALON decisions but weakly with true risk,")
print("   this confirms AVALON's misalignment!")

### 2.6 Correlation Heatmap

In [ ]:
# Select key features for correlation
key_features = [
    'core_temp_c', 'coolant_pressure_bar', 'neutron_flux',
    'radiation_inside_uSv', 'maintenance_score',
    'public_anxiety_index', 'social_media_rumour_index', 'regulator_scrutiny_score',
    'avalon_raw_risk_score', 'avalon_learned_reward_score',
    'true_risk_level', 'avalon_shutdown_recommendation', 'incident_occurred'
]

# Correlation matrix
corr_matrix = df[key_features].corr()

# Plot
plt.figure(figsize=(16, 14))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Heatmap: Key Features', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Top correlations with incident_occurred
print("\n=== TOP CORRELATIONS WITH ACTUAL INCIDENTS ===")
incident_corr = df.corr()['incident_occurred'].sort_values(ascending=False)
print(incident_corr.head(15))

### 2.7 Time Series Analysis

In [ ]:
# Yearly trends
yearly_stats = df.groupby('year').agg({
    'incident_occurred': 'mean',
    'avalon_shutdown_recommendation': 'mean',
    'true_risk_level': 'mean',
    'public_anxiety_index': 'mean'
})

fig, axes = plt.subplots(2, 2, figsize=(18, 10))
fig.suptitle('Temporal Trends (1991-2025)', fontsize=16, fontweight='bold')

axes[0, 0].plot(yearly_stats.index, yearly_stats['incident_occurred'], marker='o', color='red', linewidth=2)
axes[0, 0].set_title('Incident Rate Over Time', fontweight='bold')
axes[0, 0].set_ylabel('Incident Rate')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(yearly_stats.index, yearly_stats['avalon_shutdown_recommendation'], marker='s', color='orange', linewidth=2)
axes[0, 1].set_title('AVALON Shutdown Rate Over Time', fontweight='bold')
axes[0, 1].set_ylabel('Shutdown Rate')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].plot(yearly_stats.index, yearly_stats['true_risk_level'], marker='^', color='blue', linewidth=2)
axes[1, 0].set_title('Average True Risk Over Time', fontweight='bold')
axes[1, 0].set_ylabel('Avg True Risk')
axes[1, 0].set_xlabel('Year')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(yearly_stats.index, yearly_stats['public_anxiety_index'], marker='d', color='purple', linewidth=2)
axes[1, 1].set_title('Public Anxiety Over Time', fontweight='bold')
axes[1, 1].set_ylabel('Avg Anxiety Index')
axes[1, 1].set_xlabel('Year')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 2.8 AVALON Score Analysis

In [ ]:
# Compare AVALON's two scoring systems
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Raw vs Learned scores
axes[0].scatter(df['avalon_raw_risk_score'], df['avalon_learned_reward_score'], 
                alpha=0.4, c=df['incident_occurred'], cmap='RdYlGn_r', s=20)
axes[0].set_xlabel('AVALON Raw Risk Score')
axes[0].set_ylabel('AVALON Learned Reward Score')
axes[0].set_title('Raw Risk vs Learned Reward Score', fontweight='bold')
axes[0].plot([df['avalon_raw_risk_score'].min(), df['avalon_raw_risk_score'].max()],
             [df['avalon_raw_risk_score'].min(), df['avalon_raw_risk_score'].max()],
             'r--', alpha=0.5, label='y=x')
axes[0].legend()

# Learned score vs True risk
df.boxplot(column='avalon_learned_reward_score', by='true_risk_level', ax=axes[1])
axes[1].set_title('Learned Score by True Risk Level', fontweight='bold')
axes[1].set_xlabel('True Risk Level')
axes[1].set_ylabel('AVALON Learned Reward Score')

# Raw score vs True risk
df.boxplot(column='avalon_raw_risk_score', by='true_risk_level', ax=axes[2])
axes[2].set_title('Raw Score by True Risk Level', fontweight='bold')
axes[2].set_xlabel('True Risk Level')
axes[2].set_ylabel('AVALON Raw Risk Score')

plt.suptitle('')
plt.tight_layout()
plt.show()

print("\n=== AVALON SCORES CORRELATION ===")
print(f"Raw score vs Learned score: {df['avalon_raw_risk_score'].corr(df['avalon_learned_reward_score']):.3f}")
print(f"Raw score vs True risk: {df['avalon_raw_risk_score'].corr(df['true_risk_level']):.3f}")
print(f"Learned score vs True risk: {df['avalon_learned_reward_score'].corr(df['true_risk_level']):.3f}")
print(f"Raw score vs Incidents: {df['avalon_raw_risk_score'].corr(df['incident_occurred']):.3f}")
print(f"Learned score vs Incidents: {df['avalon_learned_reward_score'].corr(df['incident_occurred']):.3f}")

## 3. Key Insights from EDA

### Summary of Findings

In [ ]:
print("="*80)
print("KEY INSIGHTS FROM EXPLORATORY DATA ANALYSIS")
print("="*80)

print("\n1. AVALON OVERREACTION:")
print(f"   - AVALON recommends shutdown in {(df['avalon_shutdown_recommendation'].mean()*100):.1f}% of cases")
print(f"   - Only {(df['incident_occurred'].mean()*100):.1f}% of cases result in actual incidents")
print(f"   - This represents a {(df['avalon_shutdown_recommendation'].mean() / df['incident_occurred'].mean()):.1f}x overreaction rate")

print("\n2. SOCIAL BIAS:")
social_corr_avalon = df[social_bias_features].corrwith(df['avalon_shutdown_recommendation']).mean()
social_corr_true = df[social_bias_features].corrwith(df['true_risk_level']).mean()
print(f"   - Avg correlation of social features with AVALON decisions: {social_corr_avalon:.3f}")
print(f"   - Avg correlation of social features with true risk: {social_corr_true:.3f}")
print(f"   - Bias ratio: {abs(social_corr_avalon / social_corr_true):.2f}x")

print("\n3. FALSE POSITIVES:")
fp_rate = ((df['avalon_shutdown_recommendation'] == 1) & (df['incident_occurred'] == 0)).mean()
print(f"   - {(fp_rate*100):.1f}% of cases are unnecessary shutdowns")
print(f"   - This could destabilize the energy grid")

print("\n4. HUMAN TRUST:")
print(f"   - Humans override AVALON in only {df['human_override'].sum()} cases ({(df['human_override'].mean()*100):.2f}%)")
print(f"   - This suggests dangerous over-reliance on a flawed system")

print("\n5. DATA QUALITY:")
print(f"   - {df['sensor_anomaly_flag'].sum()} sensor anomalies detected ({(df['sensor_anomaly_flag'].mean()*100):.1f}%)")
print(f"   - Faulty sensors may contribute to AVALON's poor decisions")

print("\n" + "="*80)

## 4. Data Preprocessing

### 4.1 Feature Engineering

In [ ]:
# Create a copy for modeling
df_model = df.copy()

# Feature engineering
print("=== FEATURE ENGINEERING ===")

# 1. Reactor risk indicators
df_model['temp_pressure_risk'] = df_model['core_temp_c'] * df_model['coolant_pressure_bar'] / 1000
df_model['radiation_differential'] = df_model['radiation_inside_uSv'] - df_model['radiation_outside_uSv']
df_model['control_efficiency'] = df_model['control_rod_position_pct'] * df_model['neutron_flux'] / 100

# 2. Maintenance risk
df_model['maintenance_risk'] = df_model['days_since_maintenance'] / (df_model['maintenance_score'] + 1)

# 3. Social pressure index (the bias source)
df_model['social_pressure_index'] = (
    df_model['public_anxiety_index'] + 
    df_model['social_media_rumour_index'] + 
    df_model['regulator_scrutiny_score']
) / 3

# 4. Physical risk index (true indicators)
from sklearn.preprocessing import StandardScaler
physical_cols = ['core_temp_c', 'coolant_pressure_bar', 'neutron_flux', 
                 'radiation_inside_uSv', 'radiation_outside_uSv']
scaler_temp = StandardScaler()
physical_normalized = scaler_temp.fit_transform(df_model[physical_cols])
df_model['physical_risk_index'] = physical_normalized.mean(axis=1)

# 5. Age-power interaction
df_model['age_power_ratio'] = df_model['reactor_age_years'] * df_model['reactor_nominal_power_mw'] / 1000

# 6. Population risk
df_model['population_risk'] = df_model['population_within_30km'] * df_model['true_risk_level'] / 1000

print(f"New features created: {len([c for c in df_model.columns if c not in df.columns])}")
print(f"Total features now: {df_model.shape[1]}")
print("
New feature preview:")
print(df_model[['temp_pressure_risk', 'social_pressure_index', 'physical_risk_index', 
               'maintenance_risk', 'age_power_ratio']].head())